In [1]:
#We first import the necessary modules

from copy import deepcopy #we will use deepcopy for the board
import time
import math

In [2]:
#We define a class for the paws

class paw:
    
    def __init__(self, grid, move=None, comes_from = None, value = None):
        self.grid = grid
        self.move = move
        self.comes_from = comes_from
        self.value = value
    
    #We now define a function that returns all the possible next stages of the game 
    #taking into account the moves the paws can make and 
    #the fact that 'jumping' can be either a mandatory move 
    #or not in the game
    def next_stage(self, player_min, has_to_jump):
        current_stage = deepcopy(self.grid) 
        moves_can_make = []
        next_stage = [] #list that stores next_stage of the game
        #We initialise two variables that store the values of the 
        #rows where the paws get promoted for the min and the max player. 
        promotion_row = 0 
        promoted = "" 
        
        #when the minimiser (black player, starts at the top of the grid) plays:
        if player_min is True:
            moves_can_make = Draughts.look_for_moves(current_stage, has_to_jump)
            promoted = 'B'
            promotion_row = 7
        #when the maximiser (white player, starts at the bottom of the grid) plays:
        else:
            moves_can_make = Draughts.look_for_moves_player(current_stage, has_to_jump)
            promoted = 'W'
            promotion_row = 0
        for r in range(len(moves_can_make)):
            previous_row = moves_can_make[r][0]
            previous_column = moves_can_make[r][1]
            new_row = moves_can_make[r][2]
            new_column = moves_can_make[r][3]
            #we update the stage of the game
            stage = deepcopy(current_stage)
            #the paw makes a move
            Draughts.paw_makes_move(stage, previous_row, previous_column, new_row, new_column, promoted, promotion_row)
            #the next possible stages are added to the list that stores its values
            next_stage.append(paw(stage, [previous_row, previous_column, new_row, new_column]))
        return next_stage
    
    #we define functions that returns the value of the paw
    def determ_value(self, value):
        self.value = value
    def get_value(self):
        return self.value
    #these function tells us which coordinates the paw is coming from
    def set_comes_from(self, comes_from):
        self.comes_from = comes_from
    def get_comes_from(self):
        return  self.comes_from
    #we define a function to access the grid
    def get_grid(self):
        return self.grid    

In [3]:
#We define a class for the game

class Draughts:
    
    def __init__(self):
        self.game_matrix = [[],[],[],[],[],[],[],[]] #Representing the grid
        self.turn_to_play = True #This tells us when it is the turn to play
        #both players start the game with 12 paws
        self.bot_paws = 12 
        self.human_paws = 12
        #we initialise a list to store the moves that can be made
        moves_can_make = []
        self.has_to_jump = False #has to jump is set to false when the 
        #human player decides not to have jumping as a mandatory move
        
        for row in self.game_matrix:
            for r in range(8):
                row.append("---") #empty cells       
        self.bot_location()
        self.human_location() 
        
    #we put on the grid the paws of the bot player
    def bot_location(self):
        for r in range(3):
            for c in range(8):
                if (r+c)%2 == 1:
                    self.game_matrix[r][c] = ("b" + str(r) + str(c))
                    
        #we put on the grid the paws of the human player
                                        
    def human_location(self):
        for r in range(5, 8, 1):
            for c in range(8):
                if (r + c) % 2 == 1:
                    self.game_matrix[r][c] = ("w" + str(r) + str(c))

        #We define a function to show the grid to the human player
    def print_game_matrix(self):
        r = 0
        print()
        for row in self.game_matrix:
            print(r, end="  |") #we create a left border for the grid
            r +=1 #this will show the number of the row to help the player make a move
            for element in row:
                print(element, end=" ")
            print()
        print()
        for c in range(8):
            if c == 0:
                c ="     0"
            print(c, end="   ")
        print("\n") #this will show the number of the column so that it is easier fot the player to find the coordinates of the paw on the grid and make a choice
                
        #Now we make it possible for the human player to play against the bot by setting user's input
    def human_input(self):
        moves_can_make = Draughts.look_for_moves_player(self.game_matrix, self.has_to_jump)
        #if the paws of the bot are more than the ones of the human and there are no moves left for the human, the game ends
        if (len(moves_can_make)==0): 
            if self.bot_paws > self.human_paws:
                print("GAME OVER - BOT WINS")
                exit()
            else: 
                print("No moves left. The game is stopped.")
                exit()
            
        self.human_paws = 0
        self.bot_paws = 0
            
        while True: 
                
            take_paw = input("Select the paw you want to move: insert the number of the row and the number of the column separated by a comma (e.g. 4,5). \n If you want to surrender, type s")
            if (take_paw == ""):
                print("The game has been ended :)")
                exit()
            elif (take_paw =="s"):
                print("You left the game. The game has ended.")
                exit()
            put_paw = input("Where do you want to move the paw to? Insert the number of the row and the number of the column separated by a comma (e.g. 5,6).  \n If you want to surrender, type surrender")
            if (put_paw ==""):
                print("The game has been ended :)")
                exit()
            elif (put_paw == "s"):
                print("You left the game. The game has ended")
                exit()
            previous = take_paw.split(",")
            new = put_paw.split(",")
            #if the player inputs more than two digits: error message
            if len(previous) != 2 or len(new)!=2:
                print("Incorrect input. \n You must only insert two numbers in the order no of rows, no of columns separated by comma. \n The correct way of inserting the input is, for example: 4,5.")
            else:
                previous_row = previous[0]
                previous_column = previous[1]
                new_row = new[0]
                new_column = new[1]
                #if the player does not input numbers, error message
                if not previous_row.isdigit() or not previous_column.isdigit() or not new_row.isdigit() or not new_column.isdigit():
                    print("Incorrect input. \n You must only insert two numbers in the order no of rows, no of columns separated by comma. \n The correct way of inserting the input is, for example: 4,5.")
                else:
                    move = [int(previous_row), int(previous_column), int(new_row), int(new_column)]
                    if move not in moves_can_make:
                        print("This is not a move you can make :O")
                    else:
                        Draughts.paw_makes_move(self.game_matrix, int(previous_row), int(previous_column), int(new_row), int(new_column), "W", 0)
                            #We count the pieces of each player
                        for r in range(8):
                            for c in range(8):
                                if (self.game_matrix[r][c][0] == "b") or (self.game_matrix[r][c][0] == "B"):
                                    self.bot_paws += 1
                                elif (self.game_matrix[r][c][0] == "w") or (self.game_matrix[r][c][0] == "W"):
                                    self.human_paws += 1
                        break

        
        #we now define a function to find the moves available for the bot
    @staticmethod
    def look_for_moves(grid, has_to_jump):
        #we initialisse two lists for the possible moves and the jumps
        moves_can_make = []
        jumps_can_make = []
            
        for r in range(8):
            for c in range(8):
                #moves for the paws of the bot
                if grid[r][c][0] == "b":
                    if Draughts.legal_move(grid, r, c, r+1, c+1):
                        moves_can_make.append([r, c, r+1, c+1])
                    if Draughts.legal_move(grid, r, c, r+1, c-1):
                        moves_can_make.append([r, c, r+1, c-1])
                    if Draughts.legal_jump(grid, r, c, r+1, c-1, r+2, c-2):
                        jumps_can_make.append([r, c, r+1, c-1, r+2, c-2])
                    if Draughts.legal_jump(grid, r, c, r+1, c+1, r+2, c+2):
                        jumps_can_make.append([r, c, r+1, c+1, r+2, c+2])
                #moves for the promoted paws (i.e. Kings or Queens) of the bot the range of motion is larger
                elif grid[r][c][0] == "B":
                    if Draughts.legal_move(grid, r, c, r + 1, c + 1):
                        moves_can_make.append([r, c, r + 1, c + 1])
                    if Draughts.legal_move(grid, r, c, r + 1, c - 1):
                        moves_can_make.append([r, c, r + 1, c - 1])
                    if Draughts.legal_move(grid, r, c, r - 1, c - 1):
                        moves_can_make.append([r, c, r - 1, c - 1])
                    if Draughts.legal_move(grid, r, c, r - 1, c + 1):
                        moves_can_make.append([r, c, r - 1, c + 1])
                    if Draughts.legal_jump(grid, r, c, r + 1, c - 1, r + 2, c - 2):
                        jumps_can_make.append([r, c, r + 2, c - 2])
                    if Draughts.legal_jump(grid, r, c, r - 1, c - 1, r - 2, c - 2):
                        jumps_can_make.append([r, c, r - 2, c - 2])
                    if Draughts.legal_jump(grid, r, c, r - 1, c + 1, r - 2, c + 2):
                        jumps_can_make.append([r, c, r - 2, c + 2])
                    if Draughts.legal_jump(grid, r, c, r + 1, c + 1, r + 2, c + 2):
                        jumps_can_make.append([r, c, r + 2, c + 2])
            
        #in the case that the player chooses not to have mandatory jumps:
        if has_to_jump is False:
            jumps_can_make.append(moves_can_make)
            return jumps_can_make
        elif has_to_jump is True:
            if (len(jumps_can_make)==0):
                return moves_can_make #there is no additional move
            else:
                return jumps_can_make
        
    #we define a function that verifies the legality of a move for the bot
    @staticmethod
    def legal_move(grid, previous_row, previous_column, new_row, new_column):
        if (new_row > 7) or (new_row < 0): #cannot go outside the grid
            return False
        if (new_column > 7) or (new_column < 0): #cannot go outside the grid
            return False
        if (grid[previous_row][previous_column] == "---"): #if the player selects an empty cell
            return False
        if (grid[new_row][new_column] != "---"): #if the destination cell is non empty
            return False
        if (grid[previous_row][previous_column][0] == "w") or (grid[previous_row][previous_column][0]=="W"):
            #if the cell is occupied by opponent
            return False
        if (grid[new_row][new_column] == "---"): #if empty
            return True
        
        #We define a function that verifies the validity of a jump fot the bot
    @staticmethod
    def legal_jump(grid, previous_row, previous_column, new_row, new_column, passing_r, passing_c):
        if (new_row > 7) or (new_row < 0): #cannot jump outside
            return False
        if (new_column > 7) or (new_column < 0): # cannot jump outside
            return False
        if (grid[previous_row][previous_column]=="---"): #cannot start in an empty cell
            return False
        if (grid[new_row][new_column]!="---"): #cannot end jump in an occupied cell
            return False
        if (grid[previous_row][previous_column][0] == "w") or (grid[previous_row][previous_column]=="W"): #cannot jump using opponent's paws
            return False
        return True

    #We define an heuristic function for the game

    @staticmethod
    def Heu(grid):
        #initialise variables at zero
        total = 0
        p1 = 0
        p2 = 0

        for r in range(8):
            for c in range(8):
                #if there are black paws - the bot's paws
                if (grid[r][c][0] == "b") or (grid[r][c][0] == "B"):
                    #1 point for each paw
                    p1 += 1
                    #given the number of paws, we evaluate the total value by taking into account 
                    #all the possible factors we can think of
                    if (grid[r][c][0] == "b"): #normal paw
                        total += 6
                    if (grid[r][c][0]== "B"):
                        total += 12 #the King/Queen has greater weight 
                    if (r==0) or (c==0) or (r==7) or (c==7): #first position or last position
                        total += 8
                    if ((r+1) > 7) or ((c-1)<0) or ((r-1)<0) or ((c+1)>7):
                        continue
                        #if human paws are in the position to attack the bot paws
                    if ((grid[r+1][c-1][0]=="w") or (grid[r+1][c-1][0]=="W")) and (grid[r-1][c+1]=="---"):
                        total -= 3
                    if ((grid[r + 1][c + 1][0] == "w") or (grid[r + 1][c + 1] == "W")) and (grid[r - 1][c - 1] == "---"):
                        total -= 3
                        #if in a position of attack by King/Queen
                    if (grid[r - 1][c + 1][0] == "W") and (grid[r + 1][c - 1] == "---"):
                        total -= 3 
                    if (r + 2 > 7) or (c - 2 < 0):
                        continue
                        #if bot is in a position of attack
                    if ((grid[r + 1][c - 1][0] == "W") or (grid[r + 1][c - 1][0] == "w")) and (grid[r + 2][c - 2] == "---"):
                        total += 7
                    if ((r + 2) > 7) or ((c + 2) > 7):
                        continue
                    if ((grid[r + 1][c + 1][0] == "W") or (grid[r + 1][c + 1][0] == "w")) and (grid[r + 2][c + 2] == "---"):
                        total += 7
                        #for the opponent
                elif (grid[r][c][0] == "w") or (grid[r][c][0] == "W"):
                        p2 += 1

            return (total + (p1-p2) * 10000)      

    #We look for the moves of the player, in a similar fashion as above
    @staticmethod
    def look_for_moves_player(grid, has_to_jump):
        moves_can_make = []
        jumps_can_make = []    
        for r in range(8):
            for c in range(8):
                if grid[r][c][0] == "w":
                    if Draughts.legal_moves_player(grid, r, c, r - 1, c - 1):
                        moves_can_make.append([r, c, r - 1, c - 1])
                    if Draughts.legal_moves_player(grid, r, c, r - 1, c + 1):
                        moves_can_make.append([r, c, r - 1, c + 1])
                    if Draughts.legal_jump_player(grid, r, c, r - 1, c - 1, r - 2, c - 2):
                        jumps_can_make.append([r, c, r - 2, c - 2])
                    if Draughts.legal_jump_player(grid, r, c, r - 1, c + 1, r - 2, c + 2):
                        jumps_can_make.append([r, c, r - 2, c + 2])
                elif grid[r][c][0] == "W":
                    if Draughts.legal_moves_player(grid, r, c, r - 1, c - 1):
                        moves_can_make.append([r, c, r - 1, c - 1])
                    if Draughts.legal_moves_player(grid, r, c, r - 1, c + 1):
                        moves_can_make.append([r, c, r - 1, c + 1])
                    if Draughts.legal_jump_player(grid, r, c, r - 1, c - 1, r - 2, c - 2):
                        jumps_can_make.append([r, c, r - 2, c - 2])
                    if Draughts.legal_jump_player(grid, r, c, r - 1, c + 1, r - 2, c + 2):
                        jumps_can_make.append([r, c, r - 2, c + 2])
                    if Draughts.legal_moves_player(grid, r, c, r + 1, c - 1):
                        moves_can_make.append([r, c, r + 1, c - 1])
                    if Draughts.legal_jump_player(grid, r, c, r + 1, c - 1, r + 2, c - 2):
                        jumps_can_make.append([r, c, r + 2, c - 2])
                    if Draughts.legal_moves_player(grid, r, c, r + 1, c + 1):
                        moves_can_make.append([r, c, r + 1, c + 1])
                    if Draughts.legal_jump_player(grid, r, c, r + 1, c + 1, r + 2, c + 2):
                        jumps_can_make.append([r, c, r + 2, c + 2])

        if has_to_jump is False:
            jumps_can_make.append(moves_can_make)
            return jumps_can_make
        elif has_to_jump is True:
            if len(jumps_can_make) == 0:
                return moves_can_make
            else:
                return jumps_can_make

    @staticmethod
    def legal_moves_player(grid, previous_row, previous_column, new_row, new_column):
        if (new_row > 7) or (new_row < 0):
            return False
        if (new_column > 7 or new_column < 0):
            return False
        if (grid[previous_row][previous_column] == "---"):
            return False
        if (grid[new_row][new_column] != "---"):
            return False
        if (grid[previous_row][previous_column][0] == "b") or (grid[previous_row][previous_column][0] == "B"):
            return False
        if (grid[new_row][new_column] == "---"):
            return True

    @staticmethod
    def legal_jump_player(grid, previous_row, previous_column, passing_r, passing_c, new_row, new_column):
        if (new_row > 7) or (new_row < 0):
            return False
        if (new_column > 7) or (new_column < 0):
            return False
        if (grid[passing_r][passing_c] == "---"):
            return False
        if (grid[passing_r][passing_c][0] == "W") or (grid[passing_r][passing_c][0] == "w"):
            return False
        if (grid[new_row][new_column] != "---"):
            return False
        if (grid[previous_row][previous_column] == "---"):
            return False
        if (grid[previous_row][previous_column][0] == "b") or (grid[previous_row][previous_column][0] == "B"):
            return False
        return True

    #EVALUATION FUNCTION
    def game_stage_evaluation(self):
        time_1 = time.time()
        stage = paw(deepcopy(self.game_matrix))

        bot_first_moves = stage.next_stage(True, self.has_to_jump)

        if (len(bot_first_moves) == 0): #no moves left
            if (self.human_paws > self.bot_paws):
                print("Congrats! You're the winner! The bot has no moves left and you have more paws!")
                exit()
            else: 
                print("Congrats! You're the winner! The bot has no moves left!")
                exit()

        dict = {}

        for n in range(len(bot_first_moves)):
            next_ = bot_first_moves[n]
            value = Draughts.alphabeta(next_.get_grid(), 4, -math.inf, math.inf, False, self.has_to_jump)
            dict[value] = next_
        if (len(dict.keys()) == 0):
            print("Congrats! You're the winner!")

        new_grid = dict[max(dict)].get_grid()
        move = dict[max(dict)].move
        self.game_matrix = new_grid
        time_2 = time.time()
        time_3 = time_2 - time_1
        print("The bot has played its move, and it took it " + str(time_3)+ " seconds to do so!")

    #We now define a function for the MINIMAX ALGORITHM with alpha beta pruning

    def alphabeta(grid, depth, alpha, beta, max_player, has_to_jump):
        if (depth == 0):
            return Draughts.Heu(grid)
        stage = paw(deepcopy(grid))
        if max_player is True:
            maximum_alpha = -math.inf #start at minus infinite, the worst case scenario for the maximiser
            for next_ in stage.next_stage(True, has_to_jump):
                val = Draughts.alphabeta(next_.get_grid(), depth-1, alpha, beta, False, has_to_jump)
                maximum_alpha = max(maximum_alpha, val)
                alpha = max(alpha, val)
                if (alpha >= beta):
                    break
            stage.determ_value(maximum_alpha)
            return maximum_alpha
        else:
            minimum_beta = math.inf #start at plus infinite, the worst case scenario for the minimiser
            for next_ in stage.next_stage(False, has_to_jump):
                val = Draughts.alphabeta(next_.get_grid(), depth-1, alpha, beta, True, has_to_jump)
                minimum_beta = min(minimum_beta, val)
                beta = min(beta, val)
                if (alpha >= beta):
                    break
            stage.determ_value(minimum_beta)
            return minimum_beta

    @staticmethod
    def paw_makes_move(grid, previous_row, previous_column, new_row, new_column, promoted, promotion_row):
        paw_on_grid = grid[previous_row][previous_column][0]
        diff_r = previous_row - new_row
        diff_c = previous_column - new_column

        #moves on the grid leave cells emptyprintpr behind
        if diff_r == -2 and diff_c == 2:
            grid[previous_row + 1][previous_column - 1] = "---"

        elif (diff_r == 2) and (diff_c == 2):
            grid[previous_row - 1][previous_column - 1] = "---"

        elif diff_r == 2 and diff_c == -2:
            grid[previous_row - 1][previous_column + 1] = "---"

        elif diff_r == -2 and diff_c == -2:
            grid[previous_row + 1][previous_column + 1] = "---"

        #promotion to Queen/King
        if new_row == promotion_row:
            paw_on_grid = promoted
        grid[previous_row][previous_column] = "---"
        #this helps the user identifying the paws on the grid
        grid[new_row][new_column] = paw_on_grid + str(new_row) + str(new_column)

    def gameplay(self):
        print("Draughts - Human vs Machine")
        while True:
            ans = input("Would you like to play a game where jumping is a compulsory move?\n Enter: Yes or No")
            if (ans == "Yes") or (ans == "yes") or (ans =="y") or (ans == "Y"):
                self.has_to_jump = True
                break
            elif (ans == "No") or (ans == "no") or (ans == "n") or (ans =="N"):
                self.has_to_jump = False
                break
            elif (ans==""):
                print("Bye bye player! See you later!")
            else: 
                print("This program, while being amazing nonetheless, is too simple to recognise your command dear player! :)")
        while True:
            self.print_game_matrix()
            if self.turn_to_play is True:
                print("It's your turn human :)")
                self.human_input()
            else:
                print("The bot is pondering its next move...Wait")
                self.game_stage_evaluation()
            if (self.human_paws == 0):
                self.print_game_matrix()
                print("Congrats! You're the winner! The bot has no paws left, you ate all of them D:")
            elif (self.bot_paws - self.human_paws == 7):
                give_up = input("The opponent has 7 paws more than you do! Would you like to leave this game and start a new one? Enter Yes or No")
                if (give_up == "Yes") or (give_up == "yes") or (give_up =="y") or (give_up == "Y"):
                    print("Bye bye! :D")
                    exit()
            self.turn_to_play = not self.turn_to_play #end of the turn

In [ ]:
if __name__ == '__main__':
    draughts = Draughts()
    draughts.gameplay()

Draughts - Human vs Machine
Would you like to play a game where jumping is a compulsory move?
 Enter: Yes or Noy

0  |--- b01 --- b03 --- b05 --- b07 
1  |b10 --- b12 --- b14 --- b16 --- 
2  |--- b21 --- b23 --- b25 --- b27 
3  |--- --- --- --- --- --- --- --- 
4  |--- --- --- --- --- --- --- --- 
5  |w50 --- w52 --- w54 --- w56 --- 
6  |--- w61 --- w63 --- w65 --- w67 
7  |w70 --- w72 --- w74 --- w76 --- 

     0   1   2   3   4   5   6   7   

It's your turn human :)
Select the paw you want to move: insert the number of the row and the number of the column separated by a comma (e.g. 4,5). 
 If you want to surrender, type s5,2
Where do you want to move the paw to? Insert the number of the row and the number of the column separated by a comma (e.g. 5,6).  
 If you want to surrender, type surrender4,3

0  |--- b01 --- b03 --- b05 --- b07 
1  |b10 --- b12 --- b14 --- b16 --- 
2  |--- b21 --- b23 --- b25 --- b27 
3  |--- --- --- --- --- --- --- --- 
4  |--- --- --- w43 --- --- --- --- 
5 